#### Thêm thư viện cần thiết

In [10]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

#### Cấu hình trình duyệt Brave

In [11]:
options = webdriver.ChromeOptions()
options.binary_location = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
options.add_argument("--headless")  # Chạy trong chế độ headless (không hiển thị giao diện)
options.add_argument("--disable-gpu")

In [12]:
season_translation = {
    "spring": "Mùa xuân",
    "summer": "Mùa hè",
    "autumn": "Mùa thu",
    "winter": "Mùa đông"
}

#### Hàm cào thông tin anime

In [13]:
def scrape_anime_data(list_type, season, pages, output_file):
    if not os.path.exists('data/raw'):
        os.makedirs('data/raw')

    output_file_path = os.path.join('data/raw', output_file)

    season_vietnamese = season_translation.get(season, season)

    base_url = f"https://animevietsub.page/danh-sach/{list_type}/all/{season}/all/"

    driver = webdriver.Chrome(options=options)
    anime_data = []

    for i in range(1, pages + 1):
        next_page_url = f"{base_url}trang-{i}.html"
        driver.get(next_page_url)
        time.sleep(2)

        anime_elements = driver.find_elements(By.CSS_SELECTOR, "li.TPostMv")

        for element in anime_elements:
            try:
                title = element.find_element(By.CSS_SELECTOR, "h2.Title").text
                link = element.find_element(By.TAG_NAME, "a").get_attribute("href")
                image = element.find_element(By.CSS_SELECTOR, "div.Image img").get_attribute("src")
                episode = element.find_element(By.CSS_SELECTOR, "span.mli-eps").text if element.find_elements(By.CSS_SELECTOR, "span.mli-eps") else "N/A"
                rating = element.find_element(By.CSS_SELECTOR, "div.anime-avg-user-rating").get_attribute("title") if element.find_elements(By.CSS_SELECTOR, "div.anime-avg-user-rating") else "N/A"
                views = element.find_element(By.CSS_SELECTOR, "span.Year").text.replace("Lượt xem: ", "")

                quality = element.find_element(By.CSS_SELECTOR, "span.Qlty").get_attribute('innerText') if element.find_elements(By.CSS_SELECTOR, "span.Qlty") else "N/A"
                year = element.find_element(By.CSS_SELECTOR, "span.Date.AAIco-date_range").get_attribute('innerText') if element.find_elements(By.CSS_SELECTOR, "span.Date.AAIco-date_range") else "N/A"
                description = element.find_element(By.CSS_SELECTOR, "div.Description p").get_attribute('innerText') if element.find_elements(By.CSS_SELECTOR, "div.Description p") else "N/A"

                genres = ", ".join([genre.get_attribute("title") for genre in element.find_elements(By.CSS_SELECTOR, "p.Genre a")])

                anime_data.append({
                    "title": title,
                    "link": link,
                    "image": image,
                    "episode": episode,
                    "rating": rating,
                    "views": views,
                    "quality": quality,
                    "year": year,
                    "genres": genres,
                    "description": description,
                    "season": season_vietnamese
                })
            except Exception as e:
                print(f"Error extracting data for an element: {e}")

    driver.quit()

    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(anime_data, f, ensure_ascii=False, indent=4)

    print(f"Scraping completed. Data saved to {output_file_path}")

#### Crawl data

In [14]:
scrape_anime_data("list-le", "spring", 14, "anime_movies_spring.json")
scrape_anime_data("list-le", "summer", 9, "anime_movies_summer.json")
scrape_anime_data("list-le", "autumn", 11, "anime_movies_autumn.json")
scrape_anime_data("list-le", "winter", 10, "anime_movies_winter.json")
scrape_anime_data("list-bo", "spring", 47, "anime_series_spring.json")
scrape_anime_data("list-bo", "summer", 28, "anime_series_summer.json")
scrape_anime_data("list-bo", "autumn", 36, "anime_series_autumn.json")
scrape_anime_data("list-bo", "winter", 28, "anime_series_winter.json")

Scraping completed. Data saved to data/raw\anime_movies_spring.json
Scraping completed. Data saved to data/raw\anime_movies_summer.json
Scraping completed. Data saved to data/raw\anime_movies_autumn.json
Scraping completed. Data saved to data/raw\anime_movies_winter.json
Scraping completed. Data saved to data/raw\anime_series_spring.json
Scraping completed. Data saved to data/raw\anime_series_summer.json
Scraping completed. Data saved to data/raw\anime_series_autumn.json
Scraping completed. Data saved to data/raw\anime_series_winter.json
